In [5]:

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(True)

chat = ChatOpenAI(
  temperature=0.1, 
  # streaming=True, 
  # callbacks=[StreamingStdOutCallbackHandler()]
)

chat.predict("How do you make italian pasta?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [15.33s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, follow these steps:\n\n1. Boil water: Fill a large pot with water and bring it to a rolling boil. Add salt to the water for seasoning.\n\n2. Add pasta: Once the water is boiling, add the pasta to the pot. Use about 100 grams of pasta per person, or adjust according to your preference.\n\n3. Cook pasta: Cook the pasta according to the package instructions or until it reaches your desired level of doneness. Italians typically prefer their pasta cooked al dente, which means it should still have a slight bite to it.\n\n4. Make the sauce: While the pasta is cooking, prepare the sauce. There are various Italian pasta sauces you can make, such as tomato-based sauces like marinara or bolognese, cream-based sauces like Alfredo, or

"To make Italian pasta, follow these steps:\n\n1. Boil water: Fill a large pot with water and bring it to a rolling boil. Add salt to the water for seasoning.\n\n2. Add pasta: Once the water is boiling, add the pasta to the pot. Use about 100 grams of pasta per person, or adjust according to your preference.\n\n3. Cook pasta: Cook the pasta according to the package instructions or until it reaches your desired level of doneness. Italians typically prefer their pasta cooked al dente, which means it should still have a slight bite to it.\n\n4. Make the sauce: While the pasta is cooking, prepare the sauce. There are various Italian pasta sauces you can make, such as tomato-based sauces like marinara or bolognese, cream-based sauces like Alfredo, or simple olive oil and garlic-based sauces. Choose your preferred sauce and follow a recipe or make it according to your taste.\n\n5. Drain pasta: Once the pasta is cooked, carefully drain it in a colander, reserving a small amount of the cooking

In [6]:
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))

In [7]:
chat.predict("How do you make italian pasta?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [18.31s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients and steps:\n\nIngredients:\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nSteps:\n1. On a clean surface or in a large mixing bowl, create a well with the flour.\n2. Crack the eggs into the well and add salt.\n3. Using a fork, beat the eggs and gradually incorporate the flour from the sides of the well. Continue mixing until a dough starts to form.\n4. Knead the dough with your hands for about 5-10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it's too sticky, add a little flour.\n5. Once the dough is ready, cover it with a clean kitchen towel and let it rest for about 30 minutes.\n6. After

"To make Italian pasta, you will need the following ingredients and steps:\n\nIngredients:\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nSteps:\n1. On a clean surface or in a large mixing bowl, create a well with the flour.\n2. Crack the eggs into the well and add salt.\n3. Using a fork, beat the eggs and gradually incorporate the flour from the sides of the well. Continue mixing until a dough starts to form.\n4. Knead the dough with your hands for about 5-10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it's too sticky, add a little flour.\n5. Once the dough is ready, cover it with a clean kitchen towel and let it rest for about 30 minutes.\n6. After resting, divide the dough into smaller portions for easier handling.\n7. Take one portion and flatten it with a rolling pin or pasta machine until it reaches your desired thickness. You can sprinkle some flour on the surface to prevent sticking